# 제너레이터

In [26]:
# 일반적인 for문 
def index_words(text):
    result =[]
    if text:
        result.append(0)
    for ind,val in enumerate(text):
        if val == " ":
            result.append(ind+1)
    
    return result 

address = "컴퓨터(영어: Computer, 문화어: 콤퓨터, 순화어: 전산기)는 진공관"
result = index_words(address)
print(result[:10])

# 위 일반적인 for문은 핵심을 알아보기 어려움 - 가독성이 떨어짐 
# index +1의 중요성을 희석해 버림 
# 제너레이터 방법
def index_words_iter(text):
    if text:
        yield 0 
    for ind,val in enumerate(text):
        if val == " ":
            yield ind+1

# 가독성이 늘어 났다 -> 반환하는 리스트와 상호작용하는 코드가 사라짐 
# 이 함수가 호출되면 제너레이터 함수가 실제로 실행되지 않고 이터레이터를 반환한다. 
# 이터레이터가 next내장 함수를 호출할 때마다 이터레이터는 제너레이터 함수를 다음 yield까지 진행 
# 제너레이터가 yield에 전강하는 값은 이터레이터에 의해 호출하는 쪽에 반환한다.!!!! 
            
it = index_words_iter(address)
print(next(it))
print(next(it))

# 제너레이터의 출력이 이터레이터를 반환하기 때문에 손쉽게 리스트로 바꾸어 확인이 가능하다. 
result = list(index_words_iter(address))
print(result[:10])

[0, 8, 18, 23, 28, 33, 39]
0
8
[0, 8, 18, 23, 28, 33, 39]


- 또 하나의 차이점은 for문의 index_words는 모든 결과를 저장한다. 
- 제너레이터 버전으로 만들면 사용하는 메모리 크기를 어느정도 제한 가능하다. -> 입력 길이가 아무리 길어도 제일 긴 줄 정도


In [27]:
import itertools
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset +=1
            if letter == " ":
                yield offset

# 이 함수의 작업 메모리는 입력 중 가장 긴 줄의 길이로 제한 된다. 
# 제너레이터를 정의할 때 한 가지 주의함 점이 있다.
#제너레이터가 반환하는 이터레이터에 상태가 있기에 호출하는 쪽에서 재사용이 불가능 하다
with open("address.txt","r",encoding="utf-8") as f:
    it = index_file(f)
    results = itertools.islice(it,0,10)
    print(list(results))


[0, 8, 18, 23, 28, 38]


# 인자에 대해 이터레션할 때는 방어적이 돼라.
- 객체가 원소로 들어가 있는 리스트를 함수가 파라미터로 받았을 때, 이 리스트를 여러번 이터레이션 하는 것이 중요할 때가 종종 있다.


In [28]:
def normalize(numbers):
    total = sum(numbers)
    result = [] 
    for value in numbers:
        percent = 100*value /total
        result.append(percent)
    return result

visits = [15,35,80]
percentages = normalize(visits)
print(percentages)

assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [29]:
# 코드의 확장성을 높이고, 객체 수를 증가 시킨다면, 메모리가 중요하다.(훨씬 더 많은 메모리 필요)
# 그러므로 파일을 읽는 제너레이터를 구현 한다. 
def read_visits(data_path):
    with open(data_path) as f:
        for line in f: # txt 파일이지만, 'r' 옵션도 주지 않고, f.readline등의 함수도 사용하지 않는다? 
            yield int(line) # readlines랑 같은 역활?

it = read_visits("my_numbers.txt")
percentages = normalize(it)
print(percentages)

[]


- 아무 결과도 나오지 않는 이유는 이터레이터가 결과를 단 한번만 만들어내기 때문이다. 
- 이미 StopIterations 예외가 발생한 이터레이터나 제너레이터를 다시 이터레이션 하면 아무 결과도 얻을 수 없다. 

In [30]:
it = read_visits("my_numbers.txt")
print(list(it))
print(list(it))

[15, 35, 80]
[]


In [31]:
# 입력 이터레이터를 방어적으로 복사하도록 만든 코드다. 
def normalize_copy(numbers):
    numbers_copy = list(numbers)
    total = sum(numbers_copy)
    result = [] 
    for value in numbers_copy:
        percent = 100 * value / total
        result.append(percent)
    return result

it = read_visits("my_numbers.txt")
percentages = normalize_copy(it)
print(percentages)
assert sum(percentages) == 100.0

# 단 이 방법은 메모리 문제가 그대로 있다. 
# copy하면서 리스트 길이 만큼 메모리를 잡아먹기 때문이다. 
# 제너레이터를 쓰는 이유가 반감이 되어 버린다.


[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [32]:
# 이 문제를 해결하는 다른 방법으로는 호출 될 때마다 새로 이터레이터를 반환하는 함수를 받는 것이다.
def normalize_func(get_iter):
    total = sum(get_iter()) # 새 이터레이터? 
    result = [] 

    for value in get_iter(): 
        percent = 100 *value / total
        result.append(percent)
    
    return result 
path = "my_numbers.txt"
percentages = normalize_func(lambda : read_visits(path))
print(percentages)
assert sum(percentages) == 100.0
# 잘 작동하지만, 람다 함수를 넘기는 것은 보기에 좋치 않다
# 같은 결과를 달성하는 더 나은 방법은 이터레이터 프로토콜을 구현한 새로운 컨테이너 클래스를 제공하는 것이다. 


[11.538461538461538, 26.923076923076923, 61.53846153846154]



# 이터레이터 프로토콜
- 이터레이터 프로토콜은 파이썬의 For루프나 그와 관련된 식들이 컨테이너타입의 내용을 방문할 때 사용하는 절차이다. 
- 파이썬에서 for x in foo 구문을 사용하면 실제로는 iter(foo)를 호출한다.
- iter 내장함수는 foo.\_\_iter\_\_ 라는 특별 메소드를 호출한다. 
- \_\_iter\_\_메서드는 반드시 이터레이터 객체를 반환해야 한다. for 루프는 반환받은 이터레이터 객체가 데이터를 소진(StopIteration 예외)를 받을 때까지 반복적으로 이터레이터 객체에 대해 Next 내장함수를 호출한다. 


In [33]:
class ReadVisit: # 이터러블 컨테이너 타입 
    def __init__(self,data_path) -> None:
        self.data_path = data_path

    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)


- 메모리 이슈는 발생 여부가 낮아지지만, 이터레이터를 여러번 이터레이션 해야한다. 

In [34]:
visits = ReadVisit(path)
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


- 이터레이터가 iter 내장 함수에 전달되는 경우에는 전달받은 이터레이터가 그대로 반환된다.
- 컨테이너 타입이 iter에 전달되면 매번 새로운 이터레이터 객체가 반환된다. 
- 반복적으로 이터레이션 할 수 없는 인자인 경우에는 TypeError를 발생시켜서 인자를 거부한다. 

In [35]:
def normalize_defensive(numbers):
    if iter(numbers) is numbers: # 이터레이터면 그대로 이터레이터가 반환되기 때문에 즇치 않다.
        raise TypeError
    
    total = sum(numbers)
    result = [] 
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result 

pers = normalize_defensive(visits) # 컨테이너 반환 
print(pers)

per_err = normalize_defensive(read_visits(path)) # 이터레이터 반환
print(per_err)


[11.538461538461538, 26.923076923076923, 61.53846153846154]


TypeError: 

In [ ]:
from collections.abc import Iterator 

def normalize_defensive2(numbers):
    if isinstance(numbers,Iterator):
        raise TypeError
    total = sum(numbers)
    result = [] 
    for value in numbers:
        percent = 100 *value / total
        result.append(percent)
    return result


pers = normalize_defensive2(visits) # 컨테이너 반환 
print(pers)

visit = [15,35,80]
per_err2 = normalize_defensive2(visit)
print(per_err2)

[11.538461538461538, 26.923076923076923, 61.53846153846154]
[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [ ]:
per_err = normalize_defensive2(read_visits(path)) # 이터레이터 반환
print(per_err)

TypeError: 

In [ ]:
per_err4 = normalize_defensive2(iter(visit))
print(per_err4)

TypeError: 

# 리스트 컴프리헨션보다 제너레이터 식을 사용하라. 
- 각 줄에 들어 있는 문자 수를 반환한다고 하자. 
- 이를 리스트 컴프리헨션으로 하려면 파일 각 줄의 길이를 메모리에 저장해야한다. 
- 파일이 아주 크거나 절대로 끝나지 않는 네트워크 소켓이라면 리스트 컴프리헨셔을 사용하는 것은 문제가 될 수 있다. 

In [37]:
value = [len(x) for x in open("my_file.txt")]
print(value)

# 제너레이터 식을 실행해도 출력 시퀀스 전체가 실체화되지는 않는다. 대신 제너레이터 식에 들어 있는 식으로 부터 원소를 하나씩 만들어내는 이터레이터가 생성된다. 
# () 사이에 리스트 컴프리헨션과 비슷한 구문을 넣어 제너레이터 식을 만들 수 있다.
# 제너레이터 식은 이터레이터로 즉시 평가되며, 더 이상 시퀀스 원소계산이 진행되지 않는다?? 

it = (len(x) for x in open("my_file.txt"))
print(it)
print(next(it))
print(next(it))

# 제너레이터의 강력한 특징은 두 제너레이터 식을 합성할 수 있다는 점이다. 
# 다음 코드에서는 앞에서 본 제너레이터 식이 반환한 이터레이터를 다른 제너레이터 식의 입력으로 사용한다. 

Root = ((x,x**0.5) for x in it)
print(next(Root))

# 제너레이터를 함계 연결한 코드를 파이썬은 아주 빠르게 실행할 수 있다. 
# 아주 큰 입력 스트림에 대해 여러 기능을 합성한다면, 제너레이터 식을 선택하라. 다만 제너레이터가 반환하는 이터레이터에는 상태가 있기 때문에 
# 이터레이터를 한 번만 사용해야 한다. 

[100, 57, 15, 1, 12, 75, 5, 86, 89, 10]
<generator object <genexpr> at 0x13021f580>
100
57
(15, 3.872983346207417)
